In [1]:
from dotenv import load_dotenv
import os
import wandb

# load the github/ wandb api keys
load_dotenv()

True

## Getting metrics tied to the current production model
We can get the production model from the registry, then get the metrics via lineage from the associated run.

In [2]:
# init wandb client
run = wandb.init(job_type="CI/CD")
path = 'himasha/model-registry/'
model_name = 'mnist-registry'
# setup this after completing full wandb experiment
version = 'production'

artifact = run.use_artifact(f'{path}{model_name}:{version}', 
                            type='model')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: himasha. Use `wandb login --relogin` to force relogin


CommError: Project himasha/model-registry does not contain artifact: "mnist-registry:production"

In [3]:
def get_metrics(run):
    return {k:v for k,v in run.summary.items() }

In [ ]:
run = artifact.logged_by()

# get metrices tagged with the run
get_metrics(run)

## Getting metrics for a run ID
This is useful if you want to get metrics for a particular experiment ID.

In [4]:
run_path = 'himasha/mnist-experiment/2acwxb8n'
api = wandb.Api()
run = api.run(run_path)

get_metrics(run)

{'validation/loss': 0.04950812458992005,
 'validation/accuracy': 98.58999633789062,
 '_step': 250000,
 'epoch': 4,
 '_wandb': {'runtime': 25},
 '_runtime': 27.579512119293213,
 '_timestamp': 1674415573.079085,
 'train/loss': 0.02103451080620289}

## Getting metrics for a run with a tag

It's better to use the Mongo API instead of downloading and iterating through each run if possible.
The use case for this is if you wanted to compare a particular run with another run that you have tagged, such as "baseline".



In [ ]:
api = wandb.Api()
tags= ['candidate']

baseline_runs=api.runs('av-team/mlops-course-001', 
                       {"tags": {"$in": tags}}) # this is the Mongo Client


for run in baseline_runs:
    print(run.name)                 

## Promote a model to the registry
This is also might be something you could do as part of a CI/CD process.

In [5]:
#define model registry path
path = 'himasha/model-registry/mnist-registry'

run_path = 'himasha/mnist-experiment/2acwxb8n' # this is the run id
api = wandb.Api()
run = api.run(run_path)

In [19]:
run.id

'2acwxb8n'

In [6]:
# get the model from the artifacts registry and promote to the model registry
art = [a for a in run.logged_artifacts() if a.type == 'model']

# below code part related to get the model artifact tagged with run and link to a existing model registry
if art:
    assert len(art) == 1, 'More then 1 artifact of type model!'
    art[0].link(path, aliases=['new-model-demo'])

In [ ]:
# check models inthe model registry
versions = api.artifact_versions('model', path)

for v in versions:
    print(f'Model version: {v.version} with tags: {v.aliases}')

latest_model = versions[0]

# generate the model url to quick look
from urllib.parse import urlencode
query = urlencode({'selectionPath': path, 'version': latest_model.version})
registry_url = f'https://wandb.ai/{latest_model.entity}/registry/model?{query}'
registry_url

## Run Comparison
This notebook documents how to use the wandb python client to create a report. These docs describe how to create and edit a report. Make sure you click on the Python SDK tab on any

In [ ]:
import wandb
import wandb.apis.reports as wr

PROJECT = 'my-report-project'
ENTITY = 'hamelsmu'

report = wr.Report(
    entity=ENTITY,
    project=PROJECT,
    title='Compare Runs',
    description="A demo of comparing runs programatically"
)  

In [ ]:
pg = wr.PanelGrid(
    runsets=[
        wr.Runset(ENTITY, PROJECT, "Run Comparison").set_filters_with_python_expr("Name in ['bountiful-badger-2', 'dastardly-duck-4']")
    ],
    panels=[
        wr.RunComparer(diff_only='split', layout={'w': 24, 'h': 15}),
    ]
)

report.blocks = report.blocks[:1] + [pg] + report.blocks[1:]
report.save()

In [ ]:
report.url

In [18]:
import re

command = '/wandb 123456xcle'
re.search('/wandb[\s+](\S+)',command).group(1)

'123456xcle'

In [22]:
 [ str('zqthrusy'), str('2acwxb8n') ]

['zqthrusy', '2acwxb8n']

In [23]:
run.id

'2acwxb8n'